# HydroSOS Streamflow Status Product Methodology
#### Jose Valles (jose.valles.leon@gmail.com)

## One month status product

### Importing the data and finding missing dates

In [1]:
# Importing the libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
plt.style.use('classic')
%matplotlib inline

from IPython.display import HTML

sns.set()

In [2]:
class EstadoHidrologico:
    def __init__(self, station_name,values,flow_cat):
        self.station_name = station_name
        self.values = values
        self.flow_cat = flow_cat
        self.DISCHARGE_MONTHLY = None

    def cargar_datos(self):
        self.DISCHARGE_MONTHLY = pd.read_csv(f'../waterbalance/input/{self.station_name}_monthly.csv',parse_dates=['Fecha'],index_col="Fecha",dayfirst=True,na_values="NA")
        self.DISCHARGE_MONTHLY['year'] = self.DISCHARGE_MONTHLY.index.year
        self.DISCHARGE_MONTHLY['month'] = self.DISCHARGE_MONTHLY.index.month
        self.DISCHARGE_MONTHLY['water_year'] = self.DISCHARGE_MONTHLY.index.year.where(self.DISCHARGE_MONTHLY.index.month < 4, self.DISCHARGE_MONTHLY.index.year + 1)
        self.DISCHARGE_MONTHLY.index = self.DISCHARGE_MONTHLY.index.map(lambda t: t.replace(day=1))
        return self.DISCHARGE_MONTHLY

    def estado_mensual(self):
        # Climatologia
        DISCHARGE_SELECTION = self.DISCHARGE_MONTHLY[(self.DISCHARGE_MONTHLY['year'] >= 1981) & (self.DISCHARGE_MONTHLY['year'] <= 2010)]
        DISCHARGE_AVERAGE = DISCHARGE_SELECTION.groupby(DISCHARGE_SELECTION.month).mean()
        DISCHARGE_AVERAGE = DISCHARGE_AVERAGE.reindex(columns=['Caudal'])
        # Calcular indicadores
        self.DISCHARGE_MONTHLY['average_percentage'] = np.nan
        self.DISCHARGE_MONTHLY['rank_average'] = np.nan
        self.DISCHARGE_MONTHLY['non_missing'] = np.nan

        for i in range(len(self.DISCHARGE_MONTHLY)):
            # Extract the current month 
            m = self.DISCHARGE_MONTHLY.month[i]
            # Extract the current year
            y = self.DISCHARGE_MONTHLY.year[i]
            self.DISCHARGE_MONTHLY.loc[self.DISCHARGE_MONTHLY.eval('month==@m & year==@y'),'rank_average']  = self.DISCHARGE_MONTHLY.query('month==@m')['Caudal'].rank()
            self.DISCHARGE_MONTHLY.loc[self.DISCHARGE_MONTHLY.eval('month==@m & year==@y'),'non_missing']  = self.DISCHARGE_MONTHLY.query('month==@m')["Caudal"].notnull().sum()
            self.DISCHARGE_MONTHLY.loc[self.DISCHARGE_MONTHLY.eval('month==@m & year==@y'),'average_percentage'] = (self.DISCHARGE_MONTHLY['Caudal'][i] - DISCHARGE_AVERAGE.query('month == @m')["Caudal"].item()) / DISCHARGE_AVERAGE.query('month == @m')["Caudal"].item()

        self.DISCHARGE_MONTHLY['percentile'] = self.DISCHARGE_MONTHLY['rank_average']/(self.DISCHARGE_MONTHLY['non_missing']+1)

        criteria = [self.DISCHARGE_MONTHLY['percentile'].between(0.87,1.00),
            self.DISCHARGE_MONTHLY['percentile'].between(0.72,0.87),
            self.DISCHARGE_MONTHLY['percentile'].between(0.28,0.72),
            self.DISCHARGE_MONTHLY['percentile'].between(0.13,0.28),
            self.DISCHARGE_MONTHLY['percentile'].between(0.00,0.13)]

        self.DISCHARGE_MONTHLY['percentile_range'] = np.select(criteria,self.values,None)
        self.DISCHARGE_MONTHLY['flowcat'] = np.select(criteria,self.flow_cat,pd.NA)

        return self.DISCHARGE_MONTHLY
    
    def estado_trimestral(self):
        # Agregar caudales
        DISCHARGE_THREE_MONTHS = self.DISCHARGE_MONTHLY.rolling(3).apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < 0.5 else np.nan)
        # Agregar columnas
        DISCHARGE_THREE_MONTHS['startMonth'] = (DISCHARGE_THREE_MONTHS.index - pd.DateOffset(months=2)).month
        DISCHARGE_THREE_MONTHS['endMonth'] = DISCHARGE_THREE_MONTHS.index.month
        DISCHARGE_THREE_MONTHS['year'] = DISCHARGE_THREE_MONTHS.index.year
        DISCHARGE_THREE_MONTHS.index = DISCHARGE_THREE_MONTHS.index.map(lambda t: t.replace(day=1))
        # Climatologia
        DISCHARGE_SELECTION_THREE_MONTH = DISCHARGE_THREE_MONTHS[(DISCHARGE_THREE_MONTHS['year'] >= 1981) & (DISCHARGE_THREE_MONTHS['year'] < 2010)]
        DISCHARGE_AVERAGE_THREE_MONTH = DISCHARGE_SELECTION_THREE_MONTH.groupby(DISCHARGE_SELECTION_THREE_MONTH.startMonth).mean()
        DISCHARGE_AVERAGE_THREE_MONTH = DISCHARGE_AVERAGE_THREE_MONTH.reindex(columns=['Caudal'])
        # Calcular indicadores
        DISCHARGE_THREE_MONTHS['average_percentage'] = np.nan
        DISCHARGE_THREE_MONTHS['rank_average'] = np.nan
        DISCHARGE_THREE_MONTHS['non_missing'] = np.nan

        for i in range(len(DISCHARGE_THREE_MONTHS)):
            # Extract the current month 
            m = DISCHARGE_THREE_MONTHS.startMonth[i]
            # Extract the current year
            y = DISCHARGE_THREE_MONTHS.year[i]
            DISCHARGE_THREE_MONTHS.loc[DISCHARGE_THREE_MONTHS.eval('startMonth==@m & year==@y'),'rank_average']  = DISCHARGE_THREE_MONTHS.query('startMonth==@m')['Caudal'].rank()
            DISCHARGE_THREE_MONTHS.loc[DISCHARGE_THREE_MONTHS.eval('startMonth==@m & year==@y'),'non_missing']  = DISCHARGE_THREE_MONTHS.query('startMonth==@m')["Caudal"].notnull().sum()
            DISCHARGE_THREE_MONTHS.loc[DISCHARGE_THREE_MONTHS.eval('startMonth==@m & year==@y'),'average_percentage'] = (DISCHARGE_THREE_MONTHS['Caudal'][i] - DISCHARGE_AVERAGE_THREE_MONTH.query('startMonth == @m')["Caudal"].item()) / DISCHARGE_AVERAGE_THREE_MONTH.query('startMonth == @m')["Caudal"].item()

        DISCHARGE_THREE_MONTHS['percentile'] = DISCHARGE_THREE_MONTHS['rank_average']/(DISCHARGE_THREE_MONTHS['non_missing']+1)

        criteria_three_months = [DISCHARGE_THREE_MONTHS['percentile'].between(0.87,1.00),
            DISCHARGE_THREE_MONTHS['percentile'].between(0.72,0.87),
            DISCHARGE_THREE_MONTHS['percentile'].between(0.28,0.72),
            DISCHARGE_THREE_MONTHS['percentile'].between(0.13,0.28),
            DISCHARGE_THREE_MONTHS['percentile'].between(0.00,0.13)]

        DISCHARGE_THREE_MONTHS['percentile_range'] = np.select(criteria_three_months,self.values,None)
        DISCHARGE_THREE_MONTHS['flowcat'] = np.select(criteria_three_months,self.flow_cat,pd.NA)

        row_labels = {1:'JFM',
             2:'FMA',
             3:'MAM',
             4:'AMJ',
             5:'MJJ',
             6:'JJA',
             7:'JAS',
             8:'ASO',
             9:'SON',
             10:'OND',
             11:'NDE',
             12:'DEF'}
        
        DISCHARGE_THREE_MONTHS['period'] = DISCHARGE_THREE_MONTHS['startMonth'].replace(row_labels) 
        return DISCHARGE_THREE_MONTHS
    
    def estado_anual(self):
        # Agrgar caudales
        DISCHARGE_TWELVE_MONTHS = self.DISCHARGE_MONTHLY.rolling(12).apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < 0.5 else np.nan)
        DISCHARGE_TWELVE_MONTHS['startMonth'] = (DISCHARGE_TWELVE_MONTHS.index - pd.DateOffset(months=2)).month
        DISCHARGE_TWELVE_MONTHS['endMonth'] = DISCHARGE_TWELVE_MONTHS.index.month
        DISCHARGE_TWELVE_MONTHS['year'] = DISCHARGE_TWELVE_MONTHS.index.year
        DISCHARGE_TWELVE_MONTHS.index = DISCHARGE_TWELVE_MONTHS.index.map(lambda t: t.replace(day=1))
        # Climatologia
        DISCHARGE_SELECTION_TWELVE_MONTH = DISCHARGE_TWELVE_MONTHS[(DISCHARGE_TWELVE_MONTHS['year'] >= 1981) & (DISCHARGE_TWELVE_MONTHS['year'] < 2010)]
        DISCHARGE_AVERAGE_TWELVE_MONTH = DISCHARGE_SELECTION_TWELVE_MONTH.groupby(DISCHARGE_SELECTION_TWELVE_MONTH.startMonth).mean()
        DISCHARGE_AVERAGE_TWELVE_MONTH = DISCHARGE_AVERAGE_TWELVE_MONTH.reindex(columns=['Caudal'])
        # Calcular indice
        DISCHARGE_TWELVE_MONTHS['average_percentage'] = np.nan
        DISCHARGE_TWELVE_MONTHS['rank_average'] = np.nan
        DISCHARGE_TWELVE_MONTHS['non_missing'] = np.nan

        for i in range(len(DISCHARGE_TWELVE_MONTHS)):
            # Extract the current month 
            m = DISCHARGE_TWELVE_MONTHS.startMonth[i]
            # Extract the current year
            y = DISCHARGE_TWELVE_MONTHS.year[i]
            DISCHARGE_TWELVE_MONTHS.loc[DISCHARGE_TWELVE_MONTHS.eval('startMonth==@m & year==@y'),'rank_average']  = DISCHARGE_TWELVE_MONTHS.query('startMonth==@m')['Caudal'].rank()
            DISCHARGE_TWELVE_MONTHS.loc[DISCHARGE_TWELVE_MONTHS.eval('startMonth==@m & year==@y'),'non_missing']  = DISCHARGE_TWELVE_MONTHS.query('startMonth==@m')["Caudal"].notnull().sum()
            DISCHARGE_TWELVE_MONTHS.loc[DISCHARGE_TWELVE_MONTHS.eval('startMonth==@m & year==@y'),'average_percentage'] = (DISCHARGE_TWELVE_MONTHS['Caudal'][i] - DISCHARGE_AVERAGE_TWELVE_MONTH.query('startMonth == @m')["Caudal"].item()) / DISCHARGE_AVERAGE_TWELVE_MONTH.query('startMonth == @m')["Caudal"].item()
    
        DISCHARGE_TWELVE_MONTHS['percentile'] = DISCHARGE_TWELVE_MONTHS['rank_average']/(DISCHARGE_TWELVE_MONTHS['non_missing']+1)

        criteria_twelve_months = [DISCHARGE_TWELVE_MONTHS['percentile'].between(0.87,1.00),
            DISCHARGE_TWELVE_MONTHS['percentile'].between(0.72,0.87),
            DISCHARGE_TWELVE_MONTHS['percentile'].between(0.28,0.72),
            DISCHARGE_TWELVE_MONTHS['percentile'].between(0.13,0.28),
            DISCHARGE_TWELVE_MONTHS['percentile'].between(0.00,0.13)]

        DISCHARGE_TWELVE_MONTHS['percentile_range'] = np.select(criteria_twelve_months,self.values,None)
        DISCHARGE_TWELVE_MONTHS['flowcat'] = np.select(criteria_twelve_months,self.flow_cat,pd.NA)
        return DISCHARGE_TWELVE_MONTHS




In [3]:
station_name = '60'
values = ['High flow','Above normal','Normal range','Below normal','Low flow']
flow_cat = [5,4,3,2,1]

In [4]:
hydro_status = EstadoHidrologico(station_name = station_name,values=values,flow_cat=flow_cat)

In [5]:
DISCHARGE_MONTHLY = hydro_status.cargar_datos()
HTML(DISCHARGE_MONTHLY.tail(12).to_html())

,Caudal,year,month,water_year
Fecha,,,,
2023-05-01,1.324112,2023,5,2024
2023-06-01,0.508412,2023,6,2024
2023-07-01,0.185751,2023,7,2024
2023-08-01,35.723896,2023,8,2024
2023-09-01,50.252104,2023,9,2024
2023-10-01,29.884430,2023,10,2024
2023-11-01,15.254896,2023,11,2024
2023-12-01,68.302716,2023,12,2024
2024-01-01,31.180262,2024,1,2024


In [6]:
STATUS_ONE_MONTH = hydro_status.estado_mensual()
HTML(STATUS_ONE_MONTH.tail(12).to_html())

,Caudal,year,month,water_year,average_percentage,rank_average,non_missing,percentile,percentile_range,flowcat
Fecha,,,,,,,,,,
2023-05-01,1.324112,2023,5,2024,-0.979495,5.0,44.0,0.111111,Low flow,1
2023-06-01,0.508412,2023,6,2024,-0.994956,3.0,44.0,0.066667,Low flow,1
2023-07-01,0.185751,2023,7,2024,-0.998287,3.0,44.0,0.066667,Low flow,1
2023-08-01,35.723896,2023,8,2024,-0.680392,5.0,44.0,0.111111,Low flow,1
2023-09-01,50.252104,2023,9,2024,-0.468909,12.0,44.0,0.266667,Below normal,2
2023-10-01,29.884430,2023,10,2024,-0.680376,10.0,44.0,0.222222,Below normal,2
2023-11-01,15.254896,2023,11,2024,-0.741520,9.0,44.0,0.200000,Below normal,2
2023-12-01,68.302716,2023,12,2024,0.980778,38.0,44.0,0.844444,Above normal,4
2024-01-01,31.180262,2024,1,2024,0.695349,35.0,45.0,0.760870,Above normal,4


In [7]:
STATUS_THREE_MONTH = hydro_status.estado_trimestral()
HTML(STATUS_THREE_MONTH.tail(12).to_html())

C:\Users\DINAGUA\AppData\Local\Temp\ipykernel_18724\2784620046.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['percentile_range'], dtype='object')
  DISCHARGE_THREE_MONTHS = self.DISCHARGE_MONTHLY.rolling(3).apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < 0.5 else np.nan)


,Caudal,year,month,water_year,average_percentage,rank_average,non_missing,percentile,flowcat,startMonth,endMonth,percentile_range,period
Fecha,,,,,,,,,,,,,
2023-05-01,3.360144,2023,4.0,2023.666667,-0.937010,7.0,44.0,0.155556,2,3,5,Below normal,MAM
2023-06-01,1.838276,2023,5.0,2024.000000,-0.973713,3.0,44.0,0.066667,1,4,6,Low flow,AMJ
2023-07-01,0.672758,2023,6.0,2024.000000,-0.992508,3.0,44.0,0.066667,1,5,7,Low flow,MJJ
2023-08-01,12.139353,2023,7.0,2024.000000,-0.883285,3.0,44.0,0.066667,1,6,8,Low flow,JJA
2023-09-01,28.720584,2023,8.0,2024.000000,-0.716700,5.0,44.0,0.111111,1,7,9,Low flow,JAS
2023-10-01,38.620144,2023,9.0,2024.000000,-0.607236,7.0,44.0,0.155556,2,8,10,Below normal,ASO
2023-11-01,31.797144,2023,10.0,2024.000000,-0.614337,10.0,44.0,0.222222,2,9,11,Below normal,SON
2023-12-01,37.814014,2023,11.0,2024.000000,-0.404990,21.0,44.0,0.466667,3,10,12,Normal range,OND
2024-01-01,38.245958,2024,8.0,2024.000000,0.048407,28.0,44.0,0.622222,3,11,1,Normal range,NDE


In [8]:
STATUS_TWELVE_MONTH = hydro_status.estado_anual()
HTML(STATUS_TWELVE_MONTH.tail(12).to_html())

C:\Users\DINAGUA\AppData\Local\Temp\ipykernel_18724\2784620046.py:103: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['percentile_range'], dtype='object')
  DISCHARGE_TWELVE_MONTHS = self.DISCHARGE_MONTHLY.rolling(12).apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < 0.5 else np.nan)


,Caudal,year,month,water_year,average_percentage,rank_average,non_missing,percentile,flowcat,startMonth,endMonth,percentile_range
Fecha,,,,,,,,,,,,
2023-05-01,17.613484,2023,6.5,2023.166667,-0.734459,4.0,43.0,0.090909,1,3,5,Low flow
2023-06-01,17.131988,2023,6.5,2023.250000,-0.740276,4.0,43.0,0.090909,1,4,6,Low flow
2023-07-01,7.603474,2023,6.5,2023.333333,-0.884815,2.0,43.0,0.045455,1,5,7,Low flow
2023-08-01,6.959863,2023,6.5,2023.416667,-0.894312,3.0,43.0,0.068182,1,6,8,Low flow
2023-09-01,9.390756,2023,6.5,2023.500000,-0.857592,3.0,43.0,0.068182,1,7,9,Low flow
2023-10-01,11.121089,2023,6.5,2023.583333,-0.832389,3.0,43.0,0.068182,1,8,10,Low flow
2023-11-01,12.027489,2023,6.5,2023.666667,-0.819371,3.0,43.0,0.068182,1,9,11,Low flow
2023-12-01,17.586002,2023,6.5,2023.750000,-0.736312,4.0,44.0,0.088889,1,10,12,Low flow
2024-01-01,20.134797,2024,6.5,2023.833333,-0.699270,4.0,44.0,0.088889,1,11,1,Low flow


In [9]:
STATUS_ONE_MONTH = STATUS_ONE_MONTH.rename_axis('date')
STATUS_THREE_MONTH = STATUS_THREE_MONTH.rename_axis('date')
STATUS_TWELVE_MONTH = STATUS_TWELVE_MONTH.rename_axis('date')

## Export to CSV files - DINAGUA format

In [ ]:
# DISCHARGE_MONTHLY[['discharge','month','year','average_percentage','percentile','percentile_range']].to_csv(f'../output/{station_name}_one-month.csv',float_format='%.3f')

In [ ]:
# DISCHARGE_THREE_MONTHS[['discharge','startMonth','endMonth','year','average_percentage','percentile','percentile_range']].to_csv(f'../output/{station_name}_three-month.csv',float_format='%.3f')

## Export to CSV files - Demostrator format

In [12]:
STATUS_ONE_MONTH[['flowcat']].to_csv(f'../waterbalance/output/01_month/cat_{station_name}.csv',na_rep='NA')
STATUS_THREE_MONTH[['flowcat']].to_csv(f'../waterbalance/output/03_month/cat_{station_name}.csv',na_rep='NA')
STATUS_TWELVE_MONTH[['flowcat']].to_csv(f'../waterbalance/output/12_month/cat_{station_name}.csv',na_rep='NA')

## Run all the basin 

In [13]:
ALL_BASIN = pd.read_csv(f'../waterbalance/cuenca_nivel2.csv',index_col="Codigo")

In [14]:
for columna, datos in ALL_BASIN.iteritems():
    # print(columna)
    # Create model instance 
    hydro_status = EstadoHidrologico(station_name = columna,values=values,flow_cat=flow_cat)
    DISCHARGE_MONTHLY = hydro_status.cargar_datos()
    STATUS_ONE_MONTH = hydro_status.estado_mensual()
    STATUS_THREE_MONTH = hydro_status.estado_trimestral()
    STATUS_TWELVE_MONTH = hydro_status.estado_anual()
    # Exportar csv
    STATUS_ONE_MONTH[['flowcat']].to_csv(f'../waterbalance/output/01_month/cat_{columna}.csv',na_rep='NA')
    STATUS_THREE_MONTH[['flowcat']].to_csv(f'../waterbalance/output/03_month/cat_{columna}.csv',na_rep='NA')
    STATUS_TWELVE_MONTH[['flowcat']].to_csv(f'../waterbalance/output/12_month/cat_{columna}.csv',na_rep='NA')
    

C:\Users\DINAGUA\AppData\Local\Temp\ipykernel_18724\2784620046.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['percentile_range'], dtype='object')
  DISCHARGE_THREE_MONTHS = self.DISCHARGE_MONTHLY.rolling(3).apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < 0.5 else np.nan)
C:\Users\DINAGUA\AppData\Local\Temp\ipykernel_18724\2784620046.py:103: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['percentile_range'], dtype='object')
  DISCHARGE_TWELVE_MONTHS = self.DISCHARGE_MONTHLY.rolling(12).apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < 0.5 else np.nan)
C:\Users\DINAGUA\AppData\Local\Temp\ipykernel_18724\2784620046.py:50: FutureWarning: Dr